### Speech Recognition: The classical way
![speech Recognition](https://raw.githubusercontent.com/HuangYiran/Natural_Language_Processing_with_Deep_Learning/master/speech_recognition_classical.png)

### Connectionist Temporal Classification(CTC)
![CTC](https://raw.githubusercontent.com/HuangYiran/Natural_Language_Processing_with_Deep_Learning/master/CTC.png)

#### How frame predictions map to output sequences
因为一个发音可能咱多个frame，所以得到的frame predictions可能会有多个重复的字符。比如cat单词得到的frame predictions可能是cc<b>aa<b>t<b>也可能是ccc<b>aa<b>tt<b>。我们应该去掉这些重复的字母，已得到我们想要的单词（如果单词值中本来就有重复呢？？）为了确保能够方便的简化frame predictions，我们使用一个约束：you can only transition between yourself and the same symbol again or yourself and a blank。<br>
The score(log probability) of any path(各种frame predictions组合) is the sum of the scores of individual categories at the different time steps.<br>
The probability of any transcript is the sum of probabilities of all paths that correspond to that transcript.
because of dynamic programming, it is possible to compute both the log probability p(Y|X) and its gradient exactly! This gradient can be propagated to neural network whose parameters can then be adujusted by your favorite optimizer（不知道dynamic programming是什么，回头查一下？？？）<br>
测试结果，输出的句子，读的时候听起来都很不错。其错误主要集中在单词的拼写错误，以及语法错误。So if you have some way of figuring out, of ranking these different paths that you produce form your model and re-rank them by just the language model, you should get much better.(好像把错误归为原读者的问题。这样能改正spelling error吗？？)

### Sequence-to-Sequence with attention for speech
CTC的缺点：
The model makes predictions just based on the data,and once it's done with making those predictions for each frame, there's no way of adjusting that prediction.
seq2seq for speech:
![Sequence to Sequence with attention for speech](https://raw.githubusercontent.com/HuangYiran/Natural_Language_Processing_with_Deep_Learning/master/Seq2Seq_for_speech.png)
基本和前面的encoder-decoder模型一样，不过输入换了。其中存在的问题是，他的输入信号会非常的强，而实验表明，输入信号越长，其对应的错误率会越高。这里我们就需要引入attention机制。<br>
注意这个模型不同于CTC它不用$<b>$

![LAS](https://raw.githubusercontent.com/HuangYiran/Natural_Language_Processing_with_Deep_Learning/master/LAS.png)
attention的方法也基本和前面的一样，就是计算和每个隐藏层输出的score，通过score来确定attention的位置。这里用的计算score的方法是：$e=f([h_t,s])$。然后把e代入softmax中，找出对应概率最高的字符<br>
encoder的另外一个区别在于: You're used to seeing a recurring neural network, which basically proceeds at the same time step as the input. So you got an input you process with through some hidden stage. You passes through one occurent step and you move on.然而针对语音识别这个问题，当我们的输入很长的时候，将导致难以训练一个好的attention model。而这在实践中是很常见的，毕竟7秒钟的音频，就有700个frames。因此在这里我们使用hierarchical encoder。<br>
Instead of one frae of processing at every time step, you collapse neighboring frames as you feed into the next layer. What this does is that every timestep, it reduces the numbr of timesteps that you have to process and also makes the processing faster. So if you do this a few times, by the time you get to the top layer of the encoder. Your number of timesteps has been reduced significantly and your attention model is able to work a lot better.<br>
这个模型，现在存在的几个特征是：
- 同一输入，会有多个不同的输出
- Causality，生成的符号，影响attention的位置。

Limitations of LAS(Seq2seq)
- Not an online model - input must all be received before transcripts can be produced
- Attention is a computational bottleneck since every output token pays attention to every input time step
- Length of input has a big impact on accuracy

### Online Sequence to Sequence Models
视图解决前两个问题。<br>
Produce outputs as input arrive has to solve the following problem:
- When has enough information arrived that the model is confident enough to output symbols
为了实现这个任务，我们使用Neural Transducer

#### A Neural Transducer
![neural transducer](https://raw.githubusercontent.com/HuangYiran/Natural_Language_Processing_with_Deep_Learning/master/neural_transducer.png)
essence: You take the input as it comes in, at regular intervals you run a sequence to sequence model on whtat you received in the last block. Instead the encoder looks at entire input, it just look at the little block and the decoder which we call the transducer here will produce the output symbols.<br> Notice that since we've blocked up that inputs, we have this situation that you may have receive some input but you aren't able to create a output. And so now we're sort of, we need the blank symbol(call it end of block here) back again in this model. It really can be the situation, you have got a long pause.<br>
One thing of this model now is it contain causality. In CTC also had the notion of not producing any outputs. But when you produce these symbols, you did not feed back what you had produced in the past. And so it didn't have these notions where the same input can produce multiple output.<br>
This model introduces an alignment problem. What you want to know is you have to produce some symbols as output, but you don't know which chunk should these symbols be aligned to. And you have to solve that problem during learning.<br>
这里chunk和block是两个东西，输入被集中在chunk中，当chunk满的时候，会被执行encoder，等待时机，把对应的encoder的output输入到decoder中，block是指decoder，他从encoder获取信息，通过这些信息，产生想要的字符序列，当decoder生成$<e>$的时候，他会再次从encoder提取信息，辅助生成下一个单词。这就存在一个问题，你并不知道现在生成的这个单词，是来自上一个chunk中，还是这一个chunk中。
![neutral transducer2](https://raw.githubusercontent.com/HuangYiran/Natural_Language_Processing_with_Deep_Learning/master/neural_transducer2.png)
注意这里每个block输出的是一个单词，然而每个单词可以由多种表达输出，比如字母a，可以由aa<e>,a<e>,aaa<e>等输出，同样的道理也适用于单词。因此如图中第一个式子，给定x，输出a的情况，为所有a可能的组合的概率的总和。另外不同于CTC的是，这里的$<e>$是携带信息的，他影响这下一个字母的输出。图中第二个式子的意思，大概是表达，依次推出每个单词的概率，取概率和最大的单词序列。然而这里让人疑惑的是，这里说到了用beam search，而这里有个下标正好是b，难道这个表示的是beam，这就有些解释不通的，beam search应该用在每个选择单词的步骤上啊，而这在这个式子中并没有表现出来（也不知道要怎么表示就是了）。<br>

#### training
![neural transducer training](https://raw.githubusercontent.com/HuangYiran/Natural_Language_Processing_with_Deep_Learning/master/neural_transducer_training.png)
During training, when you take a gradient of that combinatorial sum, 可以得到上图中第一个式子: Give me a sample from all the probability of aligning the ouput given the input, and then i will train the log probability of that sample.<br>
It's basically this happens in cases where models are a sum of combinatorial of terms(这里是单词组合). If you want to optimize such a model you basically have to take the gradient of the log prob and the gradient of the log prob turns out to be a sum of the log probs over the posteriors of the samples. And that's the case in this model as well.
Because it's really hard to optimize so we replaced this entire really complicated procedure for optimization by giving an output symbol find the best alignment and just make that alignment better, it's kind of like a terby sort of trick.

#### finding best path
Finding best path is tricky -- beam search fails easily
<center>$\hat{y}_{1...(s+B)}=arg max_{\hat{y}_{1...(S+B)}}p(\hat{y}_{1...(S+B)}|X_{1...L,y_{1...S}})$</center>
approximate Dynamic programming -- finding best alignment
![neural transducer find best path](https://raw.githubusercontent.com/HuangYiran/Natural_Language_Processing_with_Deep_Learning/master/neural_transducer_findBestPath.png)
这个不理解。事实上，还是不清楚一个block究竟是输出一个单词还是一个字母？？感觉应该是字母吧。这样前面提到的单词，都得改成字母

### Very Deep Convolutional Encoder for LAS
就是把上面hierarchical LAS改一下，改成使用cnn提取特征。最后使用特征作为预测的基础。